# Abstractive Summarization in Urdu (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

Installing libraries.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install huggingface_hub
!pip install rouge
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

Fixes encoding error with colab.

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

Mount Google Drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Logging into the Hugging Face Hub.

In [36]:
from huggingface_hub import notebook_login

notebook_login()

Setting up directories.

In [5]:
# dir = "/content/drive/MyDrive/dataset/"
dir = "/content/drive/MyDrive/convo_proj/"
train = dir + "urdu_train.jsonl"
test = dir + "urdu_test.jsonl"
val = dir + "urdu_val.jsonl"

ntrain = dir + "/SmallerDataset/" + "urdu_train.csv"
ntest = dir + "/SmallerDataset/" + "urdu_test.csv"
nval = dir + "/SmallerDataset/" + "urdu_val.csv"

In [6]:
import torch

In [7]:
from datasets import load_dataset, Dataset
urdu_dataset = load_dataset('json', data_files={'train': train,
                                              'test': test,
                                              'val': val})

print(urdu_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 67665
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8458
    })
    val: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8458
    })
})


# New section

In [8]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Summary: {example['summary']}'")

show_samples(urdu_dataset)


'>> Title: ’ادھوری نیند دفتر میں لڑائی اور خراب رویے کا سبب‘'
'>> Summary: راٹرڈیم سکول آف مینیجمینٹ کے محقیقین کے مطابق صرف ایک رات کی ادھوری نیند دفتر میں لڑائی اور خراب رویے کا سبب بن سکتی ہے۔'

'>> Title: کوہلو: راکٹ باری کے 9 واقعات'
'>> Summary: بلوچستان کے شہر کوہلو میں آج یوم آزادی پر نامعلوم افراد نے کم سے کم نو راکٹ داغے ہیں اور دو دھماکوں کی اطلاع موصول ہوئی ہے لیکن کسی قسم کا کوئی جانی نقصان نہیں ہوا ہے۔'

'>> Title: فائر بریگیڈ نے تاخیر کی، فیکٹری مالک کا الزام'
'>> Summary: کراچی میں آتشزدگی کا شکار بننے والی فیکٹری کے مالک نے واضح کیا ہے کہ واقعے والے روز فیکٹری کے دروازے بند نہیں تھے، انہوں نے فائر بریگیڈ پر غفلت کا الزام عائد کیاہے۔'


In [9]:
urdu_dataset.reset_format()

In [10]:
urdu_news = urdu_dataset

In [11]:
from datasets import concatenate_datasets, DatasetDict

urdu_news_dataset = DatasetDict()

for split in urdu_news.keys():
    urdu_news_dataset[split] = concatenate_datasets(
        [urdu_news[split]
        ]
    )
    urdu_news_dataset[split] = urdu_news_dataset[split].shuffle(seed=42)

show_samples(urdu_news_dataset)


'>> Title: پانچ کروڑ کہاں گئے؟'
'>> Summary: وانا کےمبینہ شدت پسندوں حاجی محمد عمر، حاجی شریف خان، مولوی محمد عباس اور جاوید خان کرمزخیل نے اس بیان پر شدید ردعمل کا اظہار کیا ہے کہ فوج کی جانب سے انہیں پانچ کروڑ روپے ادا کیے گئے ہیں۔'

'>> Title: ’مانچسٹر یونائیٹڈ کے سنہری دور کا اختتام ؟‘'
'>> Summary: امریکی یونیورسٹی مشی گن کے شعبۂ معاشیات کے پروفیسر اور فٹبال کی کتاب ’سوکرنومکس‘ کے مصنف سٹیون سائمنسکی کا کہنا ہے کے فٹبال کی دنیا کے تقریباً 90 فیصد مینیجرز کا اپنی ٹیم کی ہار جیت میں عمل دخل بہت کم ہوتا ہے۔'

'>> Title: سنوکر:صالح پاکستانی یا افغانی؟'
'>> Summary: سندھ ہائی کورٹ کے چیف جسٹس صبیح الدین احمد اور جسٹس گلزار احمد پر مشتمل بینچ نے سنوکر کے کھلاڑی صالح محمد کو اس ماہ کراچی میں ہونے والی ایشین سنوکر چیمپئن شپ میں حصہ لینے سے روکنے سے متعلق پٹیشن پر نوٹس جاری کرتے ہوئے سماعت چھ جون تک ملتوی کردی ہے۔'


Removing 1 word titles.

In [12]:
urdu_news_dataset = urdu_news_dataset.filter(lambda x: len(x["title"].split()) > 2)
print(urdu_news_dataset)

Filter:   0%|          | 0/67665 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8458 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8458 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 67628
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8454
    })
    val: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8454
    })
})


Loading Model.

In [13]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
inputs = tokenizer("مجھے ہنگر گیمز پڑھنا پسند تھا!")
inputs

{'input_ids': [4952, 21505, 259, 872, 162541, 12409, 50816, 1645, 21760, 1482, 6423, 1832, 9880, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁مج',
 'ھے',
 '▁',
 'ہ',
 'نگر',
 '▁گی',
 'مز',
 '▁پ',
 'ڑھ',
 'نا',
 '▁پس',
 'ند',
 '▁تھا',
 '!',
 '</s>']

Preprocessing

In [16]:
max_input_length = 11230 + 10
max_target_length = 36 + 2


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["summary"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_datasets = urdu_news_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/67628 [00:00<?, ? examples/s]

Map:   0%|          | 0/8454 [00:00<?, ? examples/s]

Map:   0%|          | 0/8454 [00:00<?, ? examples/s]

In [18]:
generated_summary = "پانچ کروڑ کہاں گئے؟"
reference_summary = "پاچ کروڑ کہاں گئے؟"

In [19]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=37a0edf368af7aa91278c745defb1f63db97adf92281b0f02cf3d6a7073b24e1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [20]:
import evaluate
rouge_score = evaluate.load("rouge")

In [21]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

In [22]:
from rouge import Rouge

generated_summary = "پانچ کروڑ کہاں گئے؟"
reference_summary = "پنچ کروڑ کہاں گئے؟"

def calc_Rouge(generated_summary, reference_summary):
  rouge = Rouge()
  scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
  for s in scores:
    scores[s] = scores[s]['r']
  return scores
scores = calc_Rouge(generated_summary, reference_summary)
print("ROUGE Scores:", scores)


ROUGE Scores: {'rouge-1': 0.75, 'rouge-2': 0.6666666666666666, 'rouge-l': 0.75}


In [23]:
scores["rouge-1"]

0.75

In [24]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
from nltk.tokenize import sent_tokenize
import re

def extract_first_n_sentences(Summary, n=3):
    return Summary.split("۔")

def three_sentence_summary(Summary):
    return '\n'.join(extract_first_n_sentences(Summary)[:3])

print(three_sentence_summary(urdu_news_dataset["train"][1]["summary"]))

بلوچستان کے شہر کوہلو میں آج یوم آزادی پر نامعلوم افراد نے کم سے کم نو راکٹ داغے ہیں اور دو دھماکوں کی اطلاع موصول ہوئی ہے لیکن کسی قسم کا کوئی جانی نقصان نہیں ہوا ہے



In [26]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(Summary) for Summary in dataset["summary"]]
    if metric == "rouge":
      return calc_Rouge(summaries, dataset["title"])

In [27]:
import pandas as pd
metric = "rouge"
score = evaluate_baseline(urdu_news_dataset["val"], metric)
rouge_names = ["rouge-1", "rouge-2", "rouge-l"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge-1': 52.34, 'rouge-2': 22.35, 'rouge-l': 46.96}

In [28]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [37]:
from huggingface_hub import notebook_login

notebook_login()

Define Model Parameters

In [30]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 5
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps=500,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = calc_Rouge(
        decoded_preds, decoded_labels
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [32]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [33]:
tokenized_datasets = tokenized_datasets.remove_columns(
    urdu_news_dataset["train"].column_names
)

In [34]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[   916,  13360,  44134,    633,  16887,   3904,  30589,    259,  72465,
           1956,  10785,   3643,    775,    548,  49243,   2154,    775,   8764,
          17629,    259,   8799,   3707,    259,   7123,    803,    858,  29193,
           9386,  20744,   1832,    259,  39853,    973,    259,    572,   9399,
           5989,   1159,  13906,   4283,   8794,    991,   1072,    259,  11481,
           2864,  16887,   2394,   4006,      1,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0],
        [  3923,  31655,   5322,    775,   2239,   7499,   1062,    872,   6495,
            973,   1424,   1956,    259,  71301,   6864,    406,   1197,   5738,
          20697,    259,  11890,   2138,   3980,   1295,   3980,   2859,    916,
          44964,    550,  33730,    991,   3612,   1159,   2858,  40365, 214536,
           1697,    803,  44234,   3433,  16445,  32825,    406,   1539,   3207,
           4159,    259,   92

In [38]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Train model

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

Evaluation

In [40]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
0,No log,2.292018,23.785100,9.845900,22.814800


{'eval_loss': 2.292017936706543,
 'eval_rouge-1': 23.7851,
 'eval_rouge-2': 9.8459,
 'eval_rouge-l': 22.8148}

In [41]:
max_input_length = 11230 + 10
max_target_length = 36 + 2


In [66]:
def generate_summary(text_to_summarize, tokenizer, model):
    # Tokenize input
    inputs = tokenizer(text_to_summarize, return_tensors="pt",
                       max_length=max_input_length, truncation=True)

    # Move input to the same device as the model
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_input_length,
                                 num_beams=30,min_length=30, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary_text

# Example usage:


In [64]:
text_to_summarize = "بلوچستان کے شہر کوہلو میں آج یوم آزادی پر نامعلوم افراد نے کم سے کم نو راکٹ داغے ہیں اور دو دھماکوں کی اطلاع موصول ہوئی ہے لیکن کسی قسم کا کوئی جانی نقصان نہیں ہوا ہے."
summary = generate_summary(text_to_summarize, tokenizer, model)
summary

'کوہلو میں یوم آزادی پر نامعلوم افراد نے نو راکٹ داغے'

In [67]:
text_to_summarize = "سیب دنیا بھر میں پسندیدہ پھلوں میں سے ایک ہے۔ یہ ایک خوشبو دار اور لذیذ پھل ہوتا ہے جو صحت کے لئے بھی بہترین ہے۔ سیب کے انتہائی فوائد ہیں۔ اس میں وٹامنز، معدنیات اور انٹی آکسیڈنٹس پر بھرپور مقدار میں پایا جاتا ہے۔ یہ جلد کی صحت کے لئے بھی بہترین ہے اور آنکھوں کے لئے بھی مفید ہے"
summary = generate_summary(text_to_summarize, tokenizer, model)
summary

'سیب دنیا بھر میں پسندیدہ پھلوں کی صحت کے لئے بہترین اور لذیذ پھل'

In [68]:
text_to_summarize="کشمیر دنیا کی خوبصورت ترین علاقوں میں سے ایک ہے۔ یہاں کی طبیعت کی زیبائی اور پہاڑوں کی بلندیاں دل کو محسوس کراتی ہیں۔ کشمیر کا ماہول بہت خوشگوار اور سرین ہوتا ہے، جو اسے ایک خوبصورت مقام بناتا ہے۔ یہاں کے باغات، چمکیلی جھیلیں اور سرسبز پہاڑوں نے اسے جنت بنا دیا ہے۔"
summary = generate_summary(text_to_summarize, tokenizer, model)
summary

'کشمیر کا ماہول بہت خوشگوار اور سرسبز پہاڑوں کی بلندیاں دل کو محسوس کرتا ہے'

In [69]:
# Define the directory where you want to save the model
output_dir = "/content/drive/MyDrive/conv_proj/"
# Save the trained model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/conv_proj/tokenizer_config.json',
 '/content/drive/MyDrive/conv_proj/special_tokens_map.json',
 '/content/drive/MyDrive/conv_proj/spiece.model',
 '/content/drive/MyDrive/conv_proj/added_tokens.json',
 '/content/drive/MyDrive/conv_proj/tokenizer.json')